In [2]:
#yield from做的事比yield多很多，其他語言中，類似的結構稱為await
#心理模型: yield from subgen()，subgen會接手傳值給呼叫方
#yield from x運算式做的第一件事，是呼叫iter(x)來取得一個迭代器
#yield from的主要功能是開啟一個雙向通道，連結最外面的呼叫方法，與最裡面的subgenerator副產生器
#可參考 yield from subgenerator.jpg

In [4]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [8]:
def gen2():
    yield from 'AB'
    yield from range(1本利)
list(gen2())

['A', 'B', 1, 2, 3]

In [10]:
#本例展示一個代理產生器與副產生器的連結方式，代理產生器藉由yeild from連結副產生器
#亦可延伸本邏輯: 代理產生器 yield from 代理產生器 yeild from ... yeild from 副產生器

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# the subgenerator
def averager(): #計算平均
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield 
        if term is None: #關鍵的中止條件
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# the delegating generator
def grouper(results, key):
    while True:
        results[key] = yield from averager() #yield from subgenerator
# yiled from提供一個雙向的通道


# the client code, a.k.a. the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)  #啟動協程
        for value in values:
            group.send(value)
        group.send(None)  # important!終止用 #看下面註一
        #註一: 任何以send()傳給代理產生器的值，都會被直接傳給副產生器

    #print(results)  #尚未被report整理過的樣子
    report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}


if __name__ == '__main__':
    main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m
